In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Build a deep learning model with Batch Normalization
def build_model(optimizer='adam'):
    model = Sequential()
    
    # Input layer
    model.add(Flatten(input_shape=(28, 28)))
    
    # Dense layer with Batch Normalization
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    
    # Dense layer with Batch Normalization
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    
    # Output layer
    model.add(Dense(10, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# List of optimizers to try
optimizers = ['sgd', 'adam', 'rmsprop']

# Iterate through each optimizer and evaluate the model
for optimizer in optimizers:
    # Build the model
    model = build_model(optimizer)
    
    # Train the model
    history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, verbose=1)
    
    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f'\nOptimizer: {optimizer.capitalize()}')
    print(f'Test Accuracy: {test_accuracy*100:.2f}%\n')


11490434/11490434 [==============================] - 3s 0us/step
Epoch 1/10
1500/1500 [==============================] - 11s 6ms/step - loss: 0.4101 - accuracy: 0.8823 - val_loss: 0.1998 - val_accuracy: 0.9445
Epoch 2/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.2114 - accuracy: 0.9392 - val_loss: 0.1541 - val_accuracy: 0.9567
Epoch 3/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1626 - accuracy: 0.9528 - val_loss: 0.1323 - val_accuracy: 0.9623
Epoch 4/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1341 - accuracy: 0.9611 - val_loss: 0.1204 - val_accuracy: 0.9654
Epoch 5/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.1143 - accuracy: 0.9667 - val_loss: 0.1109 - val_accuracy: 0.9681
Epoch 6/10
1500/1500 [==============================] - 8s 5ms/step - loss: 0.0993 - accuracy: 0.9707 - val_loss: 0.1028 - val_accuracy: 0.9712
Epoch 7/10
1500/1500 [==============================] - 8s 6ms/step - 

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from kerastuner.tuners import RandomSearch

# Load and preprocess the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Define the model-building function
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    # Tune the number of units in the first Dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    
    # Tune the number of layers and units in each layer
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(layers.Dense(units=hp.Int(f'layer_{i}_units', min_value=32, max_value=512, step=32), activation='relu'))
    
    model.add(layers.Dense(10, activation='softmax'))
    
    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    # Choose an optimizer
    optimizer = hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd'])

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,  # Set the number of trials
    directory='keras_tuner_logs',
    project_name='fashion_mnist_tuning'
)

# Display search space summary
tuner.search_space_summary()

# Search for the best hyperparameter configuration
tuner.search(x_train, y_train, epochs=10, validation_split=0.2)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the best hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc*100:.2f}%')


Trial 10 Complete [00h 05m 39s]
val_accuracy: 0.8786666393280029

Best val_accuracy So Far: 0.8943333625793457
Total elapsed time: 00h 46m 29s
Epoch 1/10
1500/1500 [==============================] - 41s 26ms/step - loss: 0.4853 - accuracy: 0.8244 - val_loss: 0.3921 - val_accuracy: 0.8607
Epoch 2/10
1500/1500 [==============================] - 40s 27ms/step - loss: 0.3643 - accuracy: 0.8650 - val_loss: 0.3818 - val_accuracy: 0.8627
Epoch 3/10
1500/1500 [==============================] - 42s 28ms/step - loss: 0.3279 - accuracy: 0.8781 - val_loss: 0.3404 - val_accuracy: 0.8813
Epoch 4/10
1500/1500 [==============================] - 41s 28ms/step - loss: 0.3032 - accuracy: 0.8876 - val_loss: 0.3291 - val_accuracy: 0.8822
Epoch 5/10
1500/1500 [==============================] - 41s 27ms/step - loss: 0.2850 - accuracy: 0.8938 - val_loss: 0.3248 - val_accuracy: 0.8815
Epoch 6/10
1500/1500 [==============================] - 42s 28ms/step - loss: 0.2677 - accuracy: 0.8997 - val_loss: 0.3315 - va